In [1]:
# import modules

# remove FutureWarnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# import pandas
import pandas as pd
from pandas.api.types import is_numeric_dtype, is_datetime64_ns_dtype

# import regex module
import re

# import matplotlib for plotting
import matplotlib.pyplot as plt

In [ ]:
# import data

brands     = pd.read_csv('brands.csv')
orders     = pd.read_csv('orders.csv')
orderlines = pd.read_csv('orderlines.csv')
products   = pd.read_csv('products.csv')

FileNotFoundError: ignored

In [ ]:
# 'Global' variables

PRICE_CUT_OFF    = 16000

QUANTITY_CUT_OFF = 801

In [ ]:
# define description function
def get_info(file):
    df = pd.read_csv(file)
    print(f'\n{"File:" :<20} {file :^35}')
    print('='*80)
    print('='*80)
    print('\n--- INFO ---')
    print(df.info())
    print('='*80)
    print('\n--- Description ---\n', df.describe())
    print('='*80)
    print('\n--- NANS ---\n', df.isna().sum())
    print('='*80)
    print('\n--- Duplicates ---\n', df.duplicated().sum())
    print('='*80)
    print(df.head())
    print('='*80)
    print(df.tail())

    # return df

## Get info for all files

### Brands

In [ ]:
get_info('brands.csv')


File:                            brands.csv             

--- INFO ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187 entries, 0 to 186
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   short   187 non-null    object
 1   long    187 non-null    object
dtypes: object(2)
memory usage: 3.1+ KB
None

--- Description ---
        short    long
count    187     187
unique   187     181
top      8MO  Mophie
freq       1       2

--- NANS ---
 short    0
long     0
dtype: int64

--- Duplicates ---
 0
  short       long
0   8MO  8Mobility
1   ACM       Acme
2   ADN     Adonit
3   AII      Aiino
4   AKI     Akitio
    short      long
182   XOO    Xoopar
183   XRI    X-Rite
184   XTO     Xtorm
185   ZAG  ZaggKeys
186   ZEP      Zepp


#### There is nothing to fix. No nans, duplicates, or strange values.

### Products

In [ ]:
get_info('products.csv')


File:                           products.csv            

--- INFO ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19326 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          19326 non-null  object
 1   name         19326 non-null  object
 2   desc         19319 non-null  object
 3   price        19280 non-null  object
 4   promo_price  19326 non-null  object
 5   in_stock     19326 non-null  int64 
 6   type         19276 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.0+ MB
None

--- Description ---
            in_stock
count  19326.000000
mean       0.109593
std        0.312390
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000000
max        1.000000

--- NANS ---
 sku             0
name            0
desc            7
price          46
promo_price     0
in_stock        0
type           50
dtype: int64

--- Duplicates ---
 8746
       sku    

#### To be fixed:
    - drop promo_price
    - remove nans
    - remove duplicates
    - price, and type have to be converted to a numeric type
    - remove huge price values
    - keep only skus which are also in orderlines

In [ ]:
# drop promo_price
products.drop(columns=['promo_price'], axis=1, inplace=True)

In [ ]:
# null values
products = products.dropna(subset=['desc', 'price', 'type']).reset_index(drop=True)

In [ ]:
# drop duplicate rows
products.drop_duplicates(inplace=True)

In [ ]:
# function to remove periods from numbers
def remove_periods(row_value):
    if re.search('\.\d$', row_value):
        return float(row_value.replace('.', '')) / 10
    elif re.search('\.\d\d$', row_value):
        return float(row_value.replace('.', '')) / 100
    return float(row_value.replace('.', ''))

In [ ]:
# price dtype conversion
products['price'] = products.loc[:, 'price'].apply(remove_periods)

In [ ]:
# clean product type from exponential values
def clean_product_type(row_value):
    if ('E+' in row_value):
        return int(float(row_value.split('E+')[0].replace(',', '.')) * (10 ** int(row_value.split('E+')[1])))
    return row_value


products.loc[:, 'type'] = products.loc[:, 'type'].apply(clean_product_type).astype('float64')
products.loc[:, 'type'] = products.loc[:, 'type'].astype('int32')

/tmp/xpython_859560/342915536.py:8: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  products.loc[:, 'type'] = products.loc[:, 'type'].apply(clean_product_type).astype('float64')
/tmp/xpython_859560/342915536.py:9: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  products.loc[:, 'type'] = products.loc[:, 'type'].astype('int32')


In [ ]:
# remove negative or huge price values
products = products.loc[~(products.loc[:, 'price'] < 0)]
products = products.loc[~(products.loc[:, 'price'] > PRICE_CUT_OFF)]

### Orderlines

In [ ]:
get_info('orderlines.csv')


File:                          orderlines.csv           

--- INFO ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   id                293983 non-null  int64 
 1   id_order          293983 non-null  int64 
 2   product_id        293983 non-null  int64 
 3   product_quantity  293983 non-null  int64 
 4   sku               293983 non-null  object
 5   unit_price        293983 non-null  object
 6   date              293983 non-null  object
dtypes: int64(4), object(3)
memory usage: 15.7+ MB
None

--- Description ---
                  id       id_order  product_id  product_quantity
count  2.939830e+05  293983.000000    293983.0     293983.000000
mean   1.397918e+06  419999.116544         0.0          1.121126
std    1.530096e+05   66344.486479         0.0          3.396569
min    1.119109e+06  241319.000000         0.0          1

#### To be fixed:
    - dtypes for unit_price and date
    - rename id_order to order_id
    - product_id column has no useful information - delete
    - product_quantity has 999 value(s) - remove those rows
    - unit_price has strange values - remove those rows
    - remove orders with non-existing products
    - keep only orders with order_id in orders table

In [ ]:
# dtypes

# unit_price
orderlines.loc[:, 'unit_price'] = orderlines.loc[:, 'unit_price'].apply(remove_periods).astype(float)

# date
orderlines.loc[:, 'date'] = pd.to_datetime(orderlines.loc[:, 'date'])

/tmp/xpython_859560/1339809023.py:4: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  orderlines.loc[:, 'unit_price'] = orderlines.loc[:, 'unit_price'].apply(remove_periods).astype(float)
/tmp/xpython_859560/1339809023.py:7: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  orderlines.loc[:, 'date'] = pd.to_datetime(orderlines.loc[:, 'date'])


In [ ]:
# remove product_id column
orderlines.drop(columns=['product_id'], axis=1, inplace=True)

In [ ]:
# rename id_order column
orderlines.rename(columns={'id_order': 'order_id'}, inplace=True)

In [ ]:
# remove strange values from product_quantity 
orderlines = orderlines.loc[~(orderlines.loc[:, 'product_quantity'] > QUANTITY_CUT_OFF)]

In [ ]:
# remove strange values from unit_price
orderlines = orderlines.loc[~(orderlines.loc[:, 'unit_price'] < 0)]
orderlines = orderlines.loc[~(orderlines.loc[:, 'unit_price'] > PRICE_CUT_OFF)]

### Orders

In [ ]:
get_info('orders.csv')


File:                            orders.csv             

--- INFO ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226909 entries, 0 to 226908
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   order_id      226909 non-null  int64  
 1   created_date  226909 non-null  object 
 2   total_paid    226904 non-null  float64
 3   state         226909 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 6.9+ MB
None

--- Description ---
             order_id     total_paid
count  226909.000000  226904.000000
mean   413296.482480     569.225818
std     65919.250331    1761.778002
min    241319.000000       0.000000
25%    356263.000000      34.190000
50%    413040.000000     112.990000
75%    470553.000000     525.980000
max    527401.000000  214747.530000

--- NANS ---
 order_id        0
created_date    0
total_paid      5
state           0
dtype: int64

--- Duplicates ---
 0
   order_id       

#### To be fixed:
    - change dtype of created_date to date
    - remove strange values in total_paid column
    - remove nan values in total_paid column
    - keep only order_ids which are in orderlines table

In [ ]:
# dtype change to date
orders.loc[:, 'created_date'] = pd.to_datetime(orders.loc[:, 'created_date'])

/tmp/xpython_859560/1754356619.py:2: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  orders.loc[:, 'created_date'] = pd.to_datetime(orders.loc[:, 'created_date'])


In [ ]:
# remove strange prices
orders = orders.loc[~(orders.loc[:, 'total_paid'] < 0)]
orders = orders.loc[~(orders.loc[:, 'total_paid'] > PRICE_CUT_OFF)]

In [ ]:
# null values
orders = orders.dropna(subset=['total_paid']).reset_index(drop=True)

## Match skus and order_ids

In [ ]:
# Match order_id from orderlines and orders tables
orderlines = orderlines.loc[orderlines.loc[:, 'order_id'].isin(orders.loc[:, 'order_id'].unique())]
orders     = orders.loc[orders.loc[:, 'order_id'].isin(orderlines.loc[:, 'order_id'].unique())]
# print(orderlines['order_id'].nunique())
# print(orders['order_id'].nunique())

In [ ]:
# Match order_id from orderlines and orders tables
orderlines = orderlines.loc[orderlines.loc[:, 'sku'].isin(products.loc[:, 'sku'].unique())]
products   = products.loc[products.loc[:, 'sku'].isin(orderlines.loc[:, 'sku'].unique())]
# print(orderlines['sku'].nunique())
# print(products['sku'].nunique())

In [ ]:
# Create a dataframe with only completed orders
orders_full = orders.copy()
orders      = orders[orders['state'] == 'Completed'].copy()

# Final check

In [ ]:
# define description function
def get_info_df(df):
    print('='*80)
    print('\n--- INFO ---')
    print(df.info())
    print('='*80)
    print('\n--- Description ---\n', df.describe())
    print('='*80)
    print('\n--- NANS ---\n', df.isna().sum())
    print('='*80)
    print('\n--- Duplicates ---\n', df.duplicated().sum())
    print('='*80)
    print(df.head())
    print('='*80)
    print(df.tail())

In [ ]:
# Check if data is clean

def check_clean(checks):
    if not all(checks):
        print('Checks failing: ' + str([(index+1) for index, value in enumerate(checks) if not value]))
    else:
        print('Check passed')


# Products
#   - drop promo_price
#   - remove nans
#   - remove duplicates
#   - price, and type have to be converted to a numeric type
#   - remove huge price values

c1 = ('promo_price' not in products.columns)
c2 = (products.isna().sum().sum() == 0)
c3 = (products.duplicated().sum() == 0)
c4 = (is_numeric_dtype(products['price'])) & (is_numeric_dtype(products['type']))
c5 = ((products['price'].min() >= 0) & (products['price'].max() <= PRICE_CUT_OFF))

print('\nProducts:')
check_clean([c1, c2, c3, c4, c5])

# Orderlines
#   - dtypes for unit_price and date
#   - rename id_order to order_id
#   - product_id column has no useful information - delete
#   - product_quantity has 999 value(s) - remove those rows
#   - unit_price has strange values - remove those rows
#   - remove orders with non-existing products
#   - keep only orders with order_id in orders table

c1 = (is_numeric_dtype((orderlines['unit_price'])) & (is_datetime64_ns_dtype(orderlines['date'])))
c2 = (('order_id' in orderlines.columns) & ('id_order' not in orderlines.columns))
c3 = ('product_id' not in orderlines.columns)
c4 = (orderlines['product_quantity'].max() <= QUANTITY_CUT_OFF)
c5 = ((orderlines['unit_price'].min() >= 0) & (orderlines['unit_price'].max() <= PRICE_CUT_OFF))
c6 = (products['sku'].unique().sort() == orderlines['sku'].unique().sort())
c7 = (orderlines['order_id'].unique().sort() == orders['order_id'].unique().sort())

print('\nOrderlines:')
check_clean([c1, c2, c3, c4, c5, c6, c7])

# Orders
#   - change dtype of created_date to date
#   - remove strange values in total_paid column
#   - remove nan values in total_paid column
#   - keep only order_ids which are in orderlines table

c1 = (is_datetime64_ns_dtype(orders_full['created_date']))
c2 = ((orders_full['total_paid'].min() >= 0) & (orders_full['total_paid'].max() <= PRICE_CUT_OFF))
c3 = (orders_full.isna().sum().sum() == 0)
c4 = (orderlines['order_id'].unique().sort() == orders_full['order_id'].unique().sort())

print('\nOrders:')
check_clean([c1, c2, c3, c4])


Products:
Check passed

Orderlines:
Check passed

Orders:
Check passed


# Write out cleaned data

In [ ]:
# Sort dfs if necessary

brands.sort_values(by='short', ascending=True, inplace=True)
products.sort_values(by='sku', ascending=True, inplace=True)
orderlines.sort_values(by='order_id', ascending=True, inplace=True)
orders.sort_values(by='order_id', ascending=True, inplace=True)
orders_full.sort_values(by='order_id', ascending=True, inplace=True)

In [ ]:
# write out new csv files

output_directory = 'cleaned_data/'

brands_clean      = brands.to_csv(output_directory + '/brands_clean.csv', index=False)
products_clean    = products.to_csv(output_directory + '/products_clean.csv', index=False)
orderlines_clean  = orderlines.to_csv(output_directory + '/orderlines_clean.csv', index=False)
orders_clean      = orders.to_csv(output_directory + '/orders_clean.csv', index=False)
orders_full_clean = orders.to_csv(output_directory + '/orders_full_clean.csv', index=False)